# Sewershed Network Examples

In this Jupyter notebook for Python, we render examples of sewer networks based on a sewer lines in two different sewersheds.

## Data requirements

- Sewer mains
  * Sewer gravity mains and possibly force mains.
- WWTP (Wastewater treatment plants) as points

## Software setup 

We will use a couple of standard Python packages, GRASS GIS, and ImageMagick (specifically, convert).

The setup here assumes Linux. Instructions for Windows are available at [GRASS GIS Jupyter notebooks wiki page](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks#Running_a_Jupyter_notebook_locally).

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
from pathlib import Path

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

## Get the data ready

This notebooks needs two sets of gravity mains (or all mains) as vector lines and set of WWTP as points with an attribute specifying owner and facility name or at least one of them. The gravity mains file should be in directory `data/sewers_{city name}` and should be named `gravity_lines.shp` (any format or filename will work). The WWTP points need to be in one of these directories. This can be modified as needed in the code.

In [ ]:
grass_project = "data/network_examples"

To compute the data, we will use a GRASS project (aka location).

In [ ]:
!grass -e -c $data_directory $grass_project

In [ ]:
session = gj.init(grass_project)

In [ ]:
gs.run_command(
    "v.import", input="data/sewers_Raleigh/gravity_lines.shp", output="raleigh"
)

In [ ]:
gs.run_command(
    "v.import",
    input="data/sewers_Greenville/gravity_lines.shp",
    output="greenville",
)

In [ ]:
point_file = "wwtp_points"  # Filename without file extension
wwtp_vector = "wwtp"

In [ ]:
gs.run_command(
    "v.import", input="data/sewers_Greenville", layer=point_file, output=wwtp_vector
)

## Render network

In [ ]:
gs.run_command("g.region", vector="raleigh", grow="5000")
sewer_map = gj.Map(width=750, height=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(map="raleigh", legend_label="Gravity mains", color="#0067CC")
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'City of Raleigh'",
    attribute_column="FACILITY",
    xref="right",
    yref="top",
    label_color="black",
    label_size=12,
    label_bgcolor="white",
    flags="s",
)
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'City of Raleigh'",
)
sewer_map.d_text(text="(a)", color="#111111", size="6", at=(0, 95))
sewer_map.d_legend_vect(flags="b", at=(75, 19), title="Raleigh")
sewer_map.d_barscale(flags="n", at=(65, 7))
sewer_map.save(Path(".") / "raleigh_mains.png")
sewer_map.show()

In [ ]:
gs.run_command("g.region", vector="greenville", grow="5000")
sewer_map = gj.Map(width=750, height=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(map="greenville", legend_label="Gravity mains", color="#0067CC")
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'Greenville Utilities Commission'",
    attribute_column="FACILITY",
    xref="left",
    yref="top",
    label_color="black",
    label_size=12,
)
sewer_map.d_text(text="(b)", color="#111111", size="5", at=(0, 89))
sewer_map.d_legend_vect(flags="b", at=(75, 19), title="Greenville")
sewer_map.d_barscale(flags="n", at=(58, 7))
sewer_map.save(Path(".") / "greenville_mains.png")
sewer_map.show()

In [ ]:
!convert raleigh_mains.png greenville_mains.png +append mains_raleigh_greenville.png
from IPython.display import Image

Image("mains_raleigh_greenville.png")

## Render network with diameter

In [ ]:
gs.run_command("g.region", vector="greenville", grow="50")
sewer_map = gj.Map(width=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (<10in)",
    color="#0067CC",
    where="Diameter < 10",
    width=1,
)
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (<20in)",
    color="#00378C",
    where="Diameter >= 10 AND Diameter < 20",
    width=2,
)
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (>=20in)",
    color="#00073C",
    where="Diameter >= 20",
    width=3,
)
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'Greenville Utilities Commission'",
    attribute_column="FACILITY",
    xref="left",
    yref="top",
    label_color="black",
    label_size=12,
)
sewer_map.d_legend_vect(flags="b", at=(70, 19), title="Greenville")
sewer_map.d_barscale(flags="n", at=(40, 7))
sewer_map.save(Path(".") / "greenville_mains_diameter.png")
sewer_map.show()

## Render part of sewershed with diameter 

In [ ]:
gs.run_command(
    "g.region", vector="greenville", res=1, n="n-50000", s="s+25000", w="w-1000"
)
sewer_map = gj.Map(width=750, use_region=True)
sewer_map.d_background(color="white")
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (<10in)",
    color="#0067CC",
    where="Diameter < 10",
    width=1,
)
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (<20in)",
    color="#00378C",
    where="Diameter >= 10 AND Diameter < 20",
    width=2,
)
sewer_map.d_vect(
    map="greenville",
    legend_label="Gravity mains (>=20in)",
    color="#00073C",
    where="Diameter >= 20",
    width=3,
)
sewer_map.d_vect(
    map=wwtp_vector,
    legend_label="WWTP",
    color="#865A0C",
    fill_color="#FCA636",
    size=10,
    icon="basic/circle",
    where="OWNER = 'Greenville Utilities Commission'",
    attribute_column="FACILITY",
    xref="left",
    yref="top",
    label_color="black",
    label_size=12,
)
sewer_map.d_legend_vect(flags="b", at=(70, 25), title="Greenville (central part)")
sewer_map.d_barscale(flags="n", at=(65, 95), length="5", units="kilometers")
sewer_map.save(Path(".") / "greenville_mains_diameter_central.png")
sewer_map.show()